In [39]:
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
from dotenv import load_dotenv
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import os
import time
import json

In [40]:
VALIDATORS_API_KEY = os.getenv('VALIDATORS_API_KEY')

In [41]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)   
pd.set_option('display.width', 200)    
pd.set_option('display.float_format', '{:.8f}'.format)

### SOL Price (Using CoinGecko)



In [42]:
def get_sol_price():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=solana&vs_currencies=usd"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data["solana"]["usd"]
    else:
        print("Error fetching SOL price:", response.text)
        return 0

# Fetch SOL price
sol_price = get_sol_price()
df_price = pd.DataFrame({"timestamp": [pd.Timestamp.utcnow()], "sol_price_usd": [sol_price]})
print("SOL Price DataFrame:")
print(df_price.head())

SOL Price DataFrame:
                         timestamp  sol_price_usd
0 2025-05-05 12:59:25.843539+00:00   144.52000000


# Validators Details

In [43]:


# Assuming VALIDATORS is already defined
# Example: VALIDATORS = "your-secret-api-token"

NETWORK = "mainnet"  # Change to 'testnet' or 'pythnet' as needed
BASE_URL = f"https://www.validators.app/api/v1/validators/{NETWORK}.json"

HEADERS = {
    "Token": VALIDATORS_API_KEY
}

def fetch_all_validators(with_history=False):
    params = {}
    if with_history:
        params["with_history"] = "true"
    
    response = requests.get(BASE_URL, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        validators = response.json()
        print(f"Fetched {len(validators)} validators.")
        return pd.DataFrame(validators)
    else:
        print(f"Error fetching validator data: {response.status_code} - {response.text}")
        return pd.DataFrame()

# Usage
df_validators = fetch_all_validators(with_history=True)

# Display the first few rows
print(df_validators.head())


Fetched 1290 validators.
   network                                       account                             name  keybase_id                                   www_url  \
0  mainnet  CJtJMo1bwttF5nq2noEk8YbbUikXWWAHtEYuqVbtNen2  Shishaonthespot +MEV|JITO|PAL ✅              https://shishaonthespot.godaddysites.com   
1  mainnet  CtvdyHYt8cMuGVHFarV2RADfoCdnrbd8e9jAsB225uMW                     SolanaBull 🚀                                https://solanabull.pro   
2  mainnet  4SsMncJdtKiUcDtukkX15mqei7WiuQ9yvRtQrQW4reWC                        QuickNode                            https://www.quicknode.com/   
3  mainnet  9bxGPEvFjGHqpAHMkm97R5d8euFnpJ3ws83tMkTbcBUJ                      HanoiHilton                                  https://stakewiz.com   
4  mainnet  GzdpwmsqTaEK2yk2s1xdmzXEfH3P1UnKjZR7u7nSNXbi                   Blockscope.net  blockscope               https://www.blockscope.net/   

                                                                                            

In [44]:
df_validators.columns.to_list()

['network',
 'account',
 'name',
 'keybase_id',
 'www_url',
 'details',
 'avatar_url',
 'created_at',
 'updated_at',
 'admin_warning',
 'jito',
 'jito_commission',
 'stake_pools_list',
 'is_active',
 'avatar_file_url',
 'active_stake',
 'authorized_withdrawer_score',
 'commission',
 'data_center_concentration_score',
 'delinquent',
 'published_information_score',
 'root_distance_score',
 'security_report_score',
 'skipped_slot_score',
 'skipped_after_score',
 'software_version',
 'software_version_score',
 'stake_concentration_score',
 'consensus_mods_score',
 'vote_latency_score',
 'total_score',
 'vote_distance_score',
 'ip',
 'data_center_key',
 'autonomous_system_number',
 'latitude',
 'longitude',
 'data_center_host',
 'vote_account',
 'epoch_credits',
 'epoch',
 'url',
 'skipped_slots',
 'skipped_slot_percent',
 'ping_time']

In [70]:
df_validators["timestamp"] = df_validators["updated_at"]
df_validators["active_stake_SOL"] = df_validators["active_stake"] / 1e9
df_validators["active_stake_USD"] = df_validators["active_stake_SOL"] * sol_price


validators_metrics = df_validators[[ "timestamp", "name", "commission", "active_stake", "active_stake_SOL", "active_stake_USD", "skipped_slots", "skipped_slot_percent", "jito", "total_score", "ping_time","skipped_after_score", "root_distance_score", "vote_distance_score", "epoch", "epoch_credits", "skipped_slots" ]]

#validators_metrics = validators_metrics[validators_metrics['epoch'] != 782]
validators_metrics = validators_metrics.copy()
validators_metrics

,timestamp,name,commission,active_stake,active_stake_SOL,active_stake_USD,skipped_slots,skipped_slot_percent,jito,total_score,ping_time,skipped_after_score,root_distance_score,vote_distance_score,epoch,epoch_credits,skipped_slots
0,2025-05-05 08:10:05 UTC,Shishaonthespot +MEV|JITO|PAL ✅,5,17631421920894,17631.42192089,2548093.09600760,NaN,NaN,True,13,NaN,2,2,2,782.00000000,2750645.00000000,NaN
1,2025-02-03 03:40:05 UTC,SolanaBull 🚀,0,26320440363884,26320.44036388,3803830.04138852,0.00000000,0.0,True,13,NaN,2,2,2,782.00000000,2750675.00000000,0.00000000
2,2025-03-20 14:01:20 UTC,QuickNode,0,48998955237264,48998.95523726,7081329.01088939,0.00000000,0.0,True,13,NaN,2,2,2,782.00000000,2748006.00000000,0.00000000
3,2024-12-11 03:40:05 UTC,HanoiHilton,0,40151355706677,40151.35570668,5802673.92672896,0.00000000,0.0,True,13,NaN,2,2,2,782.00000000,2748461.00000000,0.00000000
4,2025-01-02 03:35:07 UTC,Blockscope.net,0,23025449586874,23025.44958687,3327637.97429503,0.00000000,0.0,True,13,NaN,2,2,2,782.00000000,2737714.00000000,0.00000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285,2024-12-10 03:35:11 UTC,topstaker,5,15586580420232,15586.58042023,2252572.60233193,0.00000000,0.0,True,0,NaN,2,2,2,782.00000000,2744074.00000000,0.00000000
1286,2025-05-04 18:47:48 UTC,None,100,100097717120,100.09771712,14466.12207818,NaN,NaN,False,0,NaN,0,2,2,782.00000000,2746770.00000000,NaN
1287,2025-01-08 03:35:08 UTC,gitbash,5,18165292531492,18165.29253149,2625248.07665122,0.00000000,0.0,True,0,NaN,2,2,2,782.00000000,2748708.00000000,0.00000000
1288,2024-10-11 03:35:07 UTC,None,100,2500000940382560,2500000.94038256,361300135.90408754,0.00000000,0.0,True,0,NaN,2,2,2,782.00000000,2748572.00000000,0.00000000


In [83]:
validators_metrics["epoch"].unique() #.sum()

array([782])

# Epoch

In [71]:
BASE_URL = 'https://www.validators.app/api/v1/epochs/mainnet.json'  # Replace with the desired network ('mainnet', 'testnet', 'pythnet')

# Headers with the API token for authentication
HEADERS = {
    'Token': VALIDATORS_API_KEY
}

# Make the API request
def get_epoch_data():
    params = {'per': 50, 'page': 1}  # Fetch up to 50 epochs, page 1 (adjust as needed)
    response = requests.get(BASE_URL, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data['epochs']
    else:
        print(f"Error fetching epoch data: {response.text}")
        return []

# Get epoch data
epoch_data = get_epoch_data()

# Convert the data to a pandas DataFrame
df_epochs = pd.DataFrame(epoch_data)

# Display the first few rows of the DataFrame
print(df_epochs.head())

   epoch  starting_slot  slots_in_epoch  network                created_at            total_rewards          total_active_stake
0    782      337824002          432000  mainnet  2025-05-04T18:04:09.000Z                      NaN                         NaN
1    781      337392000          432000  mainnet  2025-05-02T18:48:09.000Z 149600376018593.00000000 391208956635595392.00000000
2    780      336960000          432000  mainnet  2025-04-30T19:23:26.000Z 149328698516297.00000000 392498296158841984.00000000
3    779      336528001          432000  mainnet  2025-04-28T19:56:18.000Z 149221322367074.00000000 389876708020589120.00000000
4    778      336096000          432000  mainnet  2025-04-26T20:24:27.000Z 149812284369850.00000000 389896186286022464.00000000


In [82]:
# Fill NaN values in 'total_rewards' and 'total_active_stake' with 0 for epoch 782
# df_epochs.loc[(df_epochs['epoch'] == 782) & (df_epochs['total_rewards'].isna()), 'total_rewards'] = 0
# df_epochs.loc[(df_epochs['epoch'] == 782) & (df_epochs['total_active_stake'].isna()), 'total_active_stake'] = 0
# Drop rows where epoch is 782
df_epochs = df_epochs.copy()
df_epochs[df_epochs['epoch'] != 782]

df_epochs.head()


,epoch,starting_slot,slots_in_epoch,network,created_at,total_rewards,total_active_stake
0,782,337824002,432000,mainnet,2025-05-04T18:04:09.000Z,NaN,NaN
1,781,337392000,432000,mainnet,2025-05-02T18:48:09.000Z,149600376018593.00000000,391208956635595392.00000000
2,780,336960000,432000,mainnet,2025-04-30T19:23:26.000Z,149328698516297.00000000,392498296158841984.00000000
3,779,336528001,432000,mainnet,2025-04-28T19:56:18.000Z,149221322367074.00000000,389876708020589120.00000000
4,778,336096000,432000,mainnet,2025-04-26T20:24:27.000Z,149812284369850.00000000,389896186286022464.00000000


In [81]:
df_epochs["epoch"].unique() #.sum()

array([782, 781, 780, 779, 778, 777, 776, 775, 774, 773, 772, 771, 770,
       769, 768, 767, 766, 765, 764, 763, 762, 761, 760, 759, 758, 757,
       756, 755, 754, 753, 752, 751, 750, 749, 748, 747, 746, 745, 744,
       743, 742, 741, 740, 739, 738, 737, 736, 735, 734, 733])

In [73]:
print(validators_metrics["epoch"].dtype)

float64


In [74]:
print(df_epochs["epoch"].dtype)

int64


In [76]:
# Step 1: Coerce to numeric (convert strings like '782.0' or 'NaN' to proper float, invalids become NaN)
validators_metrics["epoch"] = pd.to_numeric(validators_metrics["epoch"], errors="coerce")

# Step 2: Drop rows where epoch is NaN or infinite
validators_metrics = validators_metrics[validators_metrics["epoch"].notnull()]
validators_metrics = validators_metrics[np.isfinite(validators_metrics["epoch"])]

# Step 3: Convert to integer
validators_metrics["epoch"] = validators_metrics["epoch"].astype(int)
print(validators_metrics["epoch"].dtype)

int64


In [77]:
merged_df = pd.merge(validators_metrics, df_epochs, on="epoch", how='left')

print(merged_df.head(n=100))

                  timestamp                             name  commission     active_stake  active_stake_SOL  active_stake_USD  skipped_slots skipped_slot_percent   jito  total_score  ping_time  \
0   2025-05-05 08:10:05 UTC  Shishaonthespot +MEV|JITO|PAL ✅           5   17631421920894    17631.42192089  2548093.09600760            NaN                  NaN   True           13        NaN   
1   2025-02-03 03:40:05 UTC                     SolanaBull 🚀           0   26320440363884    26320.44036388  3803830.04138852     0.00000000                  0.0   True           13        NaN   
2   2025-03-20 14:01:20 UTC                        QuickNode           0   48998955237264    48998.95523726  7081329.01088939     0.00000000                  0.0   True           13        NaN   
3   2024-12-11 03:40:05 UTC                      HanoiHilton           0   40151355706677    40151.35570668  5802673.92672896     0.00000000                  0.0   True           13        NaN   
4   2025-01-02 03:35

In [49]:
merged_df["total_rewards_SOL"] = merged_df["total_rewards"] / 1e9
merged_df["total_active_stake_SOL"] = merged_df["total_active_stake"] / 1e9

merged_df["APY"] = ((1+merged_df["total_rewards_SOL"] / merged_df["total_active_stake_SOL"]) ** 182.5 - 1) * 100




# Stake Pools

In [50]:
import requests
import pandas as pd

# Set your API key and network
#api_key = VALIDATORS_API_KEY  # Make sure this is defined earlier in your code
NETWORK = 'mainnet'  # or 'testnet', 'pythnet'

# Define the request URL and headers
url = f'https://www.validators.app/api/v1/stake-pools/{NETWORK}.json'
headers = {'Token': VALIDATORS_API_KEY}

# Make the request
response = requests.get(url, headers=headers)

# Check response
if response.status_code == 200:
    data = response.json()
    stake_pools = data.get("stake_pools", [])
    
    # Convert to DataFrame
    df_stake_pools = pd.DataFrame(stake_pools)
    print(df_stake_pools.head())
else:
    print(f"Error: {response.status_code} - {response.text}")


   id                                     authority  average_apy  average_lifetime  average_score  average_skipped_slots  average_uptime  average_validators_commission  manager_fee  deposit_fee  \
0   1  AzZRvyyMHBm8EHEksWxq4ozFL7JxLMydCDMGhqM6BVck   6.56391000              1431    12.98000000             0.03847500     23.28570000                     0.05000000   2.00000000   0.00000000   
1   2  9eG63CdHjsfhHmobHgLtESGC8GabbmRcaSpHAZrtmhco   6.69989000               720    10.40000000             0.01526420     23.92110000                     2.01000000   6.00000000   0.00000000   
2   3  HbJTxftxnXgpePCshA8FubsRj9MW4kfPscfuUfn44fnt   6.93132000               811    11.48000000             0.00068123     29.13060000                     0.00000000   5.00000000   0.00000000   
3   4   W1ZQRwUfSkDKy2oefRBUWph82Vr2zg9txWMA8RQazN5   6.62296000              1427    12.00000000             0.04166670     16.16670000                     2.66000000   5.00000000   0.00000000   
4   5  BbyX1GwU

In [51]:
df_stake_pools.columns.tolist()

['id',
 'authority',
 'average_apy',
 'average_lifetime',
 'average_score',
 'average_skipped_slots',
 'average_uptime',
 'average_validators_commission',
 'manager_fee',
 'deposit_fee',
 'withdrawal_fee',
 'name',
 'network',
 'ticker',
 'validators_count',
 'total_stake',
 'average_stake',
 'delinquent_count']

In [52]:

df_stake_pools
#df_stake_validator = df_stake_pools[["name", "average_apy", "total_stake", "average_uptime", "average_validators_commission"]]

,id,authority,average_apy,average_lifetime,average_score,average_skipped_slots,average_uptime,average_validators_commission,manager_fee,deposit_fee,withdrawal_fee,name,network,ticker,validators_count,total_stake,average_stake,delinquent_count
0,1,AzZRvyyMHBm8EHEksWxq4ozFL7JxLMydCDMGhqM6BVck,6.56391000,1431,12.98000000,0.03847500,23.28570000,0.05000000,2.00000000,0.00000000,0.03000000,Socean,mainnet,scnsol,1,3188942218,3188942218,0
1,2,9eG63CdHjsfhHmobHgLtESGC8GabbmRcaSpHAZrtmhco,6.69989000,720,10.40000000,0.01526420,23.92110000,2.01000000,6.00000000,0.00000000,0.00000000,Marinade,mainnet,msol,152,4971193080484863,32705217634768,1
2,3,HbJTxftxnXgpePCshA8FubsRj9MW4kfPscfuUfn44fnt,6.93132000,811,11.48000000,0.00068123,29.13060000,0.00000000,5.00000000,0.00000000,0.11000000,Jpool,mainnet,jsol,210,1121404724394206,5340022497115,0
3,4,W1ZQRwUfSkDKy2oefRBUWph82Vr2zg9txWMA8RQazN5,6.62296000,1427,12.00000000,0.04166670,16.16670000,2.66000000,5.00000000,0.00000000,0.00000000,Lido,mainnet,stsol,12,38937805272108,3244817106009,0
4,5,BbyX1GwUNsfbcoWwnkZDo8sqGmwNDzs2765RpjyQ1pQb,6.77727000,1269,10.00000000,0.03850000,9.00000000,5.00000000,2.00000000,0.00000000,0.30000000,DAOPool,mainnet,daosol,1,890708964662,890708964662,0
5,7,6WecYymEARvjG5ZyqkrVQ6YkhPfujNzWpSPwNKXHCbV2,6.74632000,764,11.57000000,0.02997110,21.64200000,2.19000000,5.00000000,0.00000000,0.10000000,BlazeStake,mainnet,bsol,267,1094099041223983,4097749218067,2
6,8,6iQKfEyhr3bZMotVkW6beNZz5CPAkiwvgV2CTje9pVSS,7.01151000,793,10.46000000,0.01737880,21.63000000,0.03000000,4.00000000,0.00000000,0.00000000,Jito,mainnet,jitosol,215,17553865752577955,81645887221292,0
7,9,F15nfVkJFAa3H4BaHEb6hQBnmiJZwPYioDiE1yxbc5y4,7.08696000,670,12.00000000,0.00000000,29.00000000,0.00000000,3.00000000,0.00000000,0.10000000,ZippyStake,mainnet,zippysol,1,103835128057,103835128057,0
8,10,FZEaZMmrRC3PDPFMzqooKLS2JjoyVkKNd2MkHjr7Xvyq,7.14014000,647,12.07000000,0.00751695,26.18640000,0.32000000,1.00000000,0.00000000,0.10000000,Edgevana,mainnet,edgesol,66,767582039912829,11630030907770,0
9,11,AKJt3m2xJ6ANda9adBGqb5BMrheKJSwxyCfYkLuZNmjn,6.75242000,375,11.64000000,0.01260780,24.78450000,2.49000000,5.00000000,0.10000000,0.10000000,Aero,mainnet,aerosol,116,515350097430211,4442673253708,1
